In [7]:
# Import libraries
from skyfield.api import EarthSatellite, Topos, load
from skyfield.iokit import parse_tle_file
from skyfield.api import wgs84

import datetime

In [8]:
# Create a timescale and ask the current time.
ts = load.timescale()

In [9]:
# Download the TLE file of a specific Sat

Satellite_name = "EIRSAT1"  # Name of the satellite ++++ ADD INPUT AUTOMATICALLY LATER
norad_id = "58472"  # Norad Id of the satellite ++++ ADD INPUT AUTOMATICALLY LATER

max_days = 2.0  # download again once 1 days old
name = Satellite_name + "_TLE.txt"  # custom filename, not 'gp.php'

path = "./Satellite_data/" + name  # Path to load the TLE file

url = (
    "https://celestrak.org/NORAD/elements/gp.php?CATNR=" + norad_id + "&FORMAT=TLE"
)  # Generate the url with the Sat query

if not load.exists(name) or load.days_old(name) >= max_days:
    load.download(
        url, filename=path
    )  # Download the TLE text file after 2 days under Satellite_data

[#################################] 100% EIRSAT1_TLE.txt


In [10]:
# Load the TLE file
with load.open(path) as f:
    satellites = list(parse_tle_file(f, ts))

print("Loaded", len(satellites), "satellites")

Loaded 1 satellites


In [11]:
# Load a specific satellite by number
by_number = {sat.model.satnum: sat for sat in satellites}
satellite = by_number[int(norad_id)]

In [15]:
# Define observation site : Here Estaca
lat = 48.78797485885815
long = 2.0410521528514183

# Define the min elevation of the sat in the sky to detect a visibility time

min_elevation = 30  # degrees

# Get now as utc time
t = ts.now()

antenna_site = wgs84.latlon(
    lat, long
)  # position of the antenna on ground CAN BE IMPROVED BY GPS

t0 = t  # Start time of the detection [now]
t1 = t + datetime.timedelta(
    hours=24
)  # Addition of a Delta t to analyse for a visibility time here 24 hours

t, events = satellite.find_events(
    antenna_site, t0, t1, altitude_degrees=min_elevation
)  # Find event for rise culminate and set below
event_names = (
    "rise above " + str(min_elevation) + "°",
    "culminate",
    "set below " + str(min_elevation) + "°",
)
for ti, event in zip(t, events):
    name = event_names[event]
    print(ti.utc_strftime("%Y %b %d %H:%M:%S"), name)  # Print events

2024 Nov 26 20:50:24 rise above 30°
2024 Nov 26 20:51:06 culminate
2024 Nov 26 20:51:48 set below 30°
